In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

  Using cached numpy-1.16.6-cp38-cp38-win_amd64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\user\\AppData\\Local\\Temp\\pip-uninstall-820ckzl7\\linalg\\lapack_lite.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.23.4-cp38-cp38-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.6
    Uninstalling numpy-1.16.6:
      Successfully uninstalled numpy-1.16.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.10.0 which is incompatible.
tensorboard 2.10.0 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.4 which is incompatible.
mxnet 1.7.0.post2 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.23.4 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

C:\Users\user\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.9.1


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import traceback

In [4]:
!pip install --upgrade -q pyproj

In [5]:
if __name__ == '__main__':
    whole_dataset = pd.read_excel('data/감정분류데이터셋.xlsx')
    print(whole_dataset.head())

                     wav_id                                              발화문  \
0  5ef80c48497740547350bd35             지금은 힘이 들긴 하지만 갈 곳도 마땅치 않고 우선은 견뎌봐야지.   
1  5eccae1d7e21a10eee2534b8                                 아니. 다들 일하기 바쁘니까.   
2  5efe18d3704f492ee1253da8                           얼마나 됐다고 회사에서 또 연락이 왔어!   
3  5f025d07b140144dfcfed802  그렇다고 수당을 따로 챙겨주는 것도 아니고. 이런식으로 하다간, 더 이상 못 다니지.   
4  5f03251bb140144dfcfedea5                처음부터 이렇게 늦진 않았어. 최근 들어 늦기 시작한 거야.   

      상황    1번 감정  1번 감정세기    2번 감정  2번 감정세기    3번 감정  3번 감정세기    4번 감정  \
0  angry  Disgust        1    Angry        1  sadness        1  sadness   
1  angry  Neutral        0  sadness        1  sadness        1  sadness   
2  angry    Angry        1    Angry        2    Angry        2    Angry   
3  angry    Angry        2    Angry        2    Angry        1  sadness   
4  angry  Disgust        2    Angry        1    Angry        1    Angry   

   4번감정세기    5번 감정  5번 감정세기  나이      성별  
0       1  sadness        

In [6]:
from nltk.tokenize import word_tokenize 





def data_processing(raw_data):    
    
    
    # 0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복, 6: '혐오'
    # Emotion 열에 있는 문자열을 대응되는 정수로 변환해주자
    # Hint : loc 함수를 활용하여 바꾸어 보자.
    ## 여기에 코드 작성
    raw_data.loc[(raw_data['상황'] == "fear"), '상황'] = 0  #공포 => 0
    raw_data.loc[(raw_data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
    raw_data.loc[(raw_data['상황'] == "angry"), '상황'] = 2  #분노 => 2
    raw_data.loc[(raw_data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
    raw_data.loc[(raw_data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
    raw_data.loc[(raw_data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
    raw_data.loc[(raw_data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6

    
    # 판다스의 concat을 활용하여 'document' 데이터와 'label' 데이터를 연결해보자.
    # 연결한 데이터의 이름은 processed_data라고 하자.
    ## 여기에 코드 작성
    transformed_data = raw_data['상황']
    transforemd_data2=raw_data['발화문']
    processed_data = pd.concat([transforemd_data2,transformed_data],axis=1)

    processed_data.columns = ['sentence', 'label']

    return processed_data

In [7]:
def data_to_token_ids(tokenizer, single_sentence):
    # CLS 토큰과 SEP 토큰을 문장의 시작과 끝에 붙여보자.
    special_token_added = "[CLS] " + str(single_sentence) + " [SEP]"
    
    # KoBERTTokenizer의 tokenize 함수를 활용하여 문장을 토큰화해보자.
    tokenized_text = tokenizer.tokenize(special_token_added)

    # KoBERTTokenizer의 convert_tokens_to_ids 함수를 활용하여 생성된 토큰을 숫자 형태로 바꿔주자.
    token_ids = [tokenizer.convert_tokens_to_ids(tokenized_text)]

    MAX_LEN = 64
    # pad_sequences 함수를 활용하여 문장의 빈 칸에 padding을 넣어주자.
    token_ids_padded = pad_sequences(token_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    token_ids_flatten = token_ids_padded.flatten()
    return token_ids_flatten

In [8]:
def token_ids_to_mask(token_ids):
    
    # token_id에서 0보다 큰 숫자만 유효하도록 하는 'mask' 리스트를 만들자.
    mask = [float(i>0) for i in token_ids]
    
    return mask

In [9]:
def tokenize_processed_data(tokenizer, processed_dataset):

    labels = processed_dataset['label'].to_numpy()
    
    labels = labels.astype(np.int)
    
    # list comprehension을 활용하여 processed_dataset의 'sentence' 데이터를 id값으로 토큰화하자.
    tokenized_data = [data_to_token_ids(tokenizer, processed_data) for processed_data in processed_dataset['sentence']]

    # list comprehension을 활용하여 앞서 토큰화한 데이터 id를 mask로 변환하자.
    attention_masks = [token_ids_to_mask(token_ids) for token_ids in tokenized_data]
    
    return tokenized_data, labels, attention_masks

In [10]:
def split_into_train_validation(whole_data, whole_label, whole_masks):
    print("length of whole_data : " + str(len(whole_data)))
    
    # split_into_train_test의 코드를 참조하여 data와 mask를  train을 위한 것과 validation을 위한 것으로 나누자.
    
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(whole_data,
                                                                                    whole_label, 
                                                                                    random_state=2022, 
                                                                                    test_size=0.1)
    train_masks, validation_masks, _, _ = train_test_split(whole_masks, 
                                                       whole_data,
                                                       random_state=2022, 
                                                       test_size=0.1)
    
    print("length of train_data : " + str(len(train_inputs)))
    
    return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks

In [11]:
def data_to_tensor(inputs, labels, masks):
    inputs_tensor = torch.tensor(inputs)
    labels_tensor = torch.tensor(labels)
    masks_tensor = torch.tensor(masks)
    return inputs_tensor, labels_tensor, masks_tensor

In [12]:
def tensor_to_dataloader(inputs, labels, masks, mode):
    from torch.utils.data import RandomSampler, SequentialSampler
    
    batch_size=32
    data = TensorDataset(inputs, masks, labels)
    
    if mode == "train":
      # train에 적합한 sampler을 지정하자.
        sampler = RandomSampler(data)
    else:
      # test에 적합한 sampler을 지정하자.
        sampler = SequentialSampler(data)
    
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    
    return dataloader

In [13]:
def preproc(tokenizer, whole_dataset):
    # whole_dataset을 전처리하자.
    processed_dataset = data_processing(whole_dataset)
    
    # 전처리한 전체 데이터를 토큰화하자.
    tokenized_dataset, labels, attention_masks = tokenize_processed_data(tokenizer, processed_dataset)

    # 토큰화한 train용 데이터를 train용과 validation용으로 분리하자.
    train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = split_into_train_validation(tokenized_dataset, labels, attention_masks)

    # train용, validation용 데이터 각각을 텐서로 변환하자.
    train_inputs, train_labels, train_masks = data_to_tensor(train_inputs, train_labels, train_masks)
    validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_inputs, validation_labels, validation_masks)

    # train용, validation용 텐서를 dataloader로 변환하자. 
    train_dataloader = tensor_to_dataloader(train_inputs, train_labels, train_masks, "train")
    validation_dataloader = tensor_to_dataloader(validation_inputs, validation_labels, validation_masks, "validation")

    return train_dataloader, validation_dataloader

In [16]:

def main():
    from tokenization_kobert import KoBertTokenizer

    # 전체 데이터를 불러오자.
    whole_dataset = pd.read_excel('data/감정분류데이터셋.xlsx')        

    # KoBERTTokenizer를 불러오자.
    tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
  
    train, valid = preproc(tokenizer, whole_dataset)      

In [17]:
if __name__ == '__main__':
    main()

C:\Users\user\AppData\Local\Temp\ipykernel_20352\3071738246.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = labels.astype(np.int)


length of whole_data : 29991
length of train_data : 26991


C:\Users\user\AppData\Local\Temp\ipykernel_20352\663155787.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  inputs_tensor = torch.tensor(inputs)
